### Неделя 6 Дополнительные признаки

In [1]:
%pylab inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
from itertools import product
from sklearn import cross_validation, linear_model, metrics, ensemble
import scipy.cluster.hierarchy as sch

Populating the interactive namespace from numpy and matplotlib


C:\Users\JB\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\JB\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


1. Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте QmayQ_{may}Qmay​ — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016

In [2]:
# обучаться будем с 01.2016 по 04.2016
train_from = pd.to_datetime('2016-01-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

In [3]:
# подготовил предварительно данные по 102 ячейкам за январь 2014 - июнь 2016
data = pd.read_csv('data2016_m1_m6_102cell.csv', sep=';', header=0, index_col=['date_hour'], 
                  parse_dates=['date_hour'], dayfirst=True)

In [4]:
data.head(2)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date_hour,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29


In [5]:
data.tail(2)

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date_hour,,,,,,,,,,,,,,,,,,,,,
2016-06-30 22:00:00,106,168,103,125,317,476,405,508,578,259,...,3,19,5,358,387,169,12,206,146,0
2016-06-30 23:00:00,85,130,86,113,256,428,483,531,631,210,...,9,0,7,323,110,147,38,173,119,0


In [6]:
data.shape

(4368, 102)

In [7]:
# сделаю копию выборки и буду с ней экспериментировать
dt_1 = data.copy()
dt_1.reset_index(inplace=True) # выведем date_hour из индекса в обычный столбец
dt_1.shape

(4368, 103)

In [8]:
dt_1.head(2)

,date_hour,1075,1076,1077,1125,1126,1127,1128,1129,1130,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
0,2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,...,12,0,2,44,5,41,4,70,7,66
1,2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,...,29,0,5,2,2,4,0,47,1,29


In [9]:
%%time
# разверну данные в длину по зонам
dt_long1 = pd.DataFrame(columns=['cell', 'date_hour','val'])
for i in xrange(1, len(dt_1.columns)):
    dt_temp = pd.DataFrame(columns=['cell', 'date_hour','val'])
    dt_temp['date_hour'] = dt_1.date_hour.values
    dt_temp['val'] = dt_1[dt_1.columns[i]].values
    dt_temp['cell'] = [dt_1.columns[i]] * dt_1.shape[0]
    
    dt_long1 = dt_long1.append(dt_temp)

Wall time: 1.97 s


In [10]:
print dt_long1.shape
dt_long1.head(3)

(445536, 3)


,cell,date_hour,val
0,1075,2016-01-01 00:00:00,80
1,1075,2016-01-01 01:00:00,91
2,1075,2016-01-01 02:00:00,90


In [11]:
%%time
# Добавляем признаки дня, месяца, дня недели, часа и года.
dt_long1['day'] = dt_long1.date_hour.map(lambda x: x.day)
dt_long1['month'] = dt_long1.date_hour.map(lambda x: x.month)
dt_long1['weekday'] = dt_long1.date_hour.map(lambda x: x.weekday())
dt_long1['hour'] = dt_long1.date_hour.map(lambda x: x.hour)
dt_long1['year'] = dt_long1.date_hour.map(lambda x: x.year)
# Добавляем признаки числа поездок из этого региона за прошлые часы и дни
for i in xrange(1, 32): dt_long1['sh_' + str(i)] = dt_long1['val'] - dt_long1['val'].shift(i)
for i in xrange(1, 15): dt_long1['sd_' + str(i)] = dt_long1['val'] - dt_long1['val'].shift(24*i)   
#dt_long1['sum_p_12_h'] = [dt_long1['val'][i-12:i].sum() for i in xrange(dt_long1.shape[0])]
#dt_long1['sum_p_24_h'] = [dt_long1['val'][i-24:i].sum() for i in xrange(dt_long1.shape[0])]
#dt_long1['sum_p_w'] = [dt_long1['val'][i-24*7:i].sum() for i in xrange(dt_long1.shape[0])]

# суммарное количество поездок за предшествующие полдня
dt_long1['prev_12h_sum'] = dt_long1.val.rolling(12).sum()
# суммарное количество поездок за предшествующий день
dt_long1['prev_24h_sum'] = dt_long1.val.rolling(24).sum()
# суммарное количество поездок за предшествующую неделю
dt_long1['prev_Week_sum'] = dt_long1.val.rolling(168).sum()


# Добавляем целевые значения для каждой из 6 моделей
for i in xrange(1, 7): dt_long1['target_' + str(i)] = dt_long1['val'].shift(-i)
   

Wall time: 6.83 s


In [12]:
# вот что получилось по признакам
#dt_long1.fillna(0,inplace=True)
dt_long1.dropna(inplace=True)
print dt_long1.shape
dt_long1.head(4) 

(445194, 62)


,cell,date_hour,val,day,month,weekday,hour,year,sh_1,sh_2,...,sd_14,prev_12h_sum,prev_24h_sum,prev_Week_sum,target_1,target_2,target_3,target_4,target_5,target_6
336,1075,2016-01-15 00:00:00,44,15,1,4,0,2016,-16,-59,...,-36,1021.0,1547.0,9016.0,30,13,6,3,2,19
337,1075,2016-01-15 01:00:00,30,15,1,4,1,2016,-14,-30,...,-61,971.0,1559.0,9027.0,13,6,3,2,19,36
338,1075,2016-01-15 02:00:00,13,15,1,4,2,2016,-17,-31,...,-77,891.0,1561.0,9031.0,6,3,2,19,36,78
339,1075,2016-01-15 03:00:00,6,15,1,4,3,2016,-7,-24,...,-26,813.0,1563.0,9028.0,3,2,19,36,78,70


In [ ]:
#чистка лишних признакоы
#for i in range(1):
#    dt_long1.drop(dt_long1.columns[len(dt_long1.columns)-1], inplace=True, axis=1)

In [48]:
# SIN COS не стал добавлять, ухудшают работу лин-й регрессии
#%%time
# Создадим регрессионные признаки для учёта сезонностей и трендов.
#Si=sin([1,…,T]∗2πi/168),Ci=cos([1,…,T]∗2πi/168),i=1,…,K.
# К возьму 7
#itog_param = []
#temp_param = []
#K=7
#arr_param = np.zeros((445194,14), dtype=float)

#задублирую на все зоны
#for j in range(K):
#    temp_param = []
#    for i in range(len(arr_param)):    
#        arr_param[i,j*2] = sin((i+1)*2*np.pi*(j+1)/168.)
#        arr_param[i,j*2+1] = cos((i+1)*2*np.pi*(j+1)/168.)
        
# добавляем синусы и косинусы в итоговый фрейм
#for i in range(arr_param.shape[1]/2):
#    dt_long1['sin_'+str(i+1)] = arr_param[:,i*2]
#    dt_long1['cos_'+str(i+1)] = arr_param[:,i*2+1]

Wall time: 6.28 s


In [13]:
# подготовлю тест и трейн
X_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]
X_train.drop(['weekday','val','date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]
X_test.drop(['weekday','val','date_hour','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)


C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [14]:
%%time
# Create linear regression object для для 6-ти концов истории своя модель
# заодно посчитаю ошибку за май
models = dict()
Qmay = 0 # ошибка предсказания за май

for i in xrange(1,7):

    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]['target_'+str(i)].values
    y_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]['target_'+str(i)].values
    
    clf_LIN = linear_model.LinearRegression()
    clf_LIN.fit(X_train, y_train)
    pred_LIN = clf_LIN.predict(X_test)
    score_LIN = clf_LIN.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_LIN[pred_LIN < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_LIN)
    
    Qmay += sum(abs(y_test-pred_LIN))
    
    print 'точность модели linear regression для Т+'+str(i)+' = '+ str(score_LIN)+' Q=' + str(1/(102*739.*i)*Qmay) 

точность модели linear regression для Т+1 = 0.971247695145 Q=20.9442050748
точность модели linear regression для Т+2 = 0.939615816278 Q=24.8860278308
точность модели linear regression для Т+3 = 0.918638211134 Q=27.5130414925
точность модели linear regression для Т+4 = 0.906647575342 Q=29.3042643991
точность модели linear regression для Т+5 = 0.901692433869 Q=30.4939703541
точность модели linear regression для Т+6 = 0.900545615498 Q=31.2961194475
Wall time: 15.9 s


In [15]:
# считаем ошибку прогноза
1/(102*739*6.)*Qmay

31.296119447519295

In [17]:
%%time
# попробую RandomForest
Qmay_mae = 0 # ошибка предсказания за май
Qmay = 0 # ошибка предсказания за май

for i in xrange(1,7):

    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_long1[(dt_long1.date_hour >= train_from) & (dt_long1.date_hour <= train_to)]['target_'+str(i)].values
    y_test = dt_long1[(dt_long1.date_hour >= test_from) & (dt_long1.date_hour <= test_to)]['target_'+str(i)].values
    
    clf_RF = ensemble.RandomForestRegressor(n_estimators=70,max_features=30,n_jobs=16)
    clf_RF.fit(X_train, y_train)
    pred_RF = clf_RF.predict(X_test)
    score_RF = clf_RF.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_RF[pred_LIN < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_RF)
    
    Qmay += sum(abs(y_test-pred_RF))
    Qmay_mae += metrics.mean_absolute_error(y_test, pred_RF)  
    
    print 'точность модели linear regression для Т+'+str(i)+' = '+ str(score_RF)+' Q=' + str(1/(102*739.*i)*Qmay) 
    print 'MAE = ' + str(Qmay_mae/i)   

точность модели linear regression для Т+1 = 0.983534546076 Q=15.6707620261
MAE = 15.6707620261
точность модели linear regression для Т+2 = 0.97430455028 Q=17.1855904337
MAE = 17.1855904337
точность модели linear regression для Т+3 = 0.969007249402 Q=18.2376825245
MAE = 18.2376825245
точность модели linear regression для Т+4 = 0.965473004548 Q=19.0024867051
MAE = 19.0024867051
точность модели linear regression для Т+5 = 0.963357372282 Q=19.5598521357
MAE = 19.5598521357
точность модели linear regression для Т+6 = 0.962798471268 Q=19.9761247756
MAE = 19.9761247756
Wall time: 8min 32s


2 Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке. 

на 6-й неделе посмотрел влияние признаков на регрессию по графику весов, оказалось что значение поездок в момент времени Т и день недели ухудшают прогноз, после их удаления ошибка стала меньше на 1,5% 

In [ ]:
#plt.figure(figsize=(16,10))
#plt.barh(X_test_tmp.columns, models[1][1].coef_, align='center',log=False)
#plt.show()

подготовил заранее файл с признаками из сырых данных:
1. fare_amount 	Время и расстояние проезда рассчитывается по счетчику
2. passenger_count 	Количество пассажиров
3. total_amount 	Общая сумма, взимаемая с пассажиров
4. trip_distance    Прошедшее расстояние в милях сообщает таксометр

за 01-06.2016, попробую за счет них улучшить регрессию

In [18]:
dt_feat = pd.read_csv('feat_2016-01-06.csv', sep=';', header=0)

In [19]:
print dt_feat.shape
#dt_feat.date_hour.astype(str)
#dt_long1.date_hour.astype(str)
dt_feat.head(2)

(499697, 6)


,date_hour,cell,fare_amount,passenger_count,total_amount,trip_distance
0,2016-01-01 00:00:00,1075,1190.5,163,1407.25,317.81
1,2016-01-01 00:00:00,1076,2119.5,301,2667.25,566.41


In [20]:
dt_feat['dt_cell'] = dt_feat.date_hour.astype(str) + '_' + dt_feat.cell.astype(str)
dt_long1['dt_cell'] = dt_long1.date_hour.astype(str) + '_' + dt_long1.cell.astype(str)

In [67]:
#dt_feat.set_index(['date_hour','cell'],inplace=True)
#dt_long1.set_index(['date_hour','cell'],inplace=True)
#dt_long1.reset_index(inplace=True) # выведем date_hour из индекса в обычный столбец
#dt_feat.reset_index(inplace=True) # выведем date_hour из индекса в обычный столбец

In [21]:
dt_long1.head(2) 

,cell,date_hour,val,day,month,weekday,hour,year,sh_1,sh_2,...,prev_12h_sum,prev_24h_sum,prev_Week_sum,target_1,target_2,target_3,target_4,target_5,target_6,dt_cell
336,1075,2016-01-15 00:00:00,44,15,1,4,0,2016,-16,-59,...,1021.0,1547.0,9016.0,30,13,6,3,2,19,2016-01-15 00:00:00_1075
337,1075,2016-01-15 01:00:00,30,15,1,4,1,2016,-14,-30,...,971.0,1559.0,9027.0,13,6,3,2,19,36,2016-01-15 01:00:00_1075


In [22]:
# сливаем фреймы по дате и ячейке
dt_new_feat = dt_long1.merge(dt_feat, how='left', left_on=['dt_cell'],right_on=['dt_cell'])

In [23]:
print dt_new_feat.shape
dt_new_feat.tail(2) 

(445194, 69)


,cell_x,date_hour_x,val,day,month,weekday,hour,year,sh_1,sh_2,...,target_4,target_5,target_6,dt_cell,date_hour_y,cell_y,fare_amount,passenger_count,total_amount,trip_distance
445192,2168,2016-06-30 16:00:00,2,30,6,3,16,2016,2,2,...,0,1,0,2016-06-30 16:00:00_2168,2016-06-30 16:00:00,2168.0,104.0,2.0,150.44,0.86
445193,2168,2016-06-30 17:00:00,1,30,6,3,17,2016,-1,1,...,1,0,0,2016-06-30 17:00:00_2168,2016-06-30 17:00:00,2168.0,52.0,6.0,57.30,0.01


In [24]:
# вот что получилось по признакам
dt_new_feat.fillna(0,inplace=True)
dt_new_feat.drop(['date_hour_y','cell_y', 'dt_cell'], inplace=True, axis=1)
#dt_new_feat.columns[0] = 'date_hour'
print dt_new_feat.shape
dt_new_feat.head(3) 

(445194, 66)


,cell_x,date_hour_x,val,day,month,weekday,hour,year,sh_1,sh_2,...,target_1,target_2,target_3,target_4,target_5,target_6,fare_amount,passenger_count,total_amount,trip_distance
0,1075,2016-01-15 00:00:00,44,15,1,4,0,2016,-16,-59,...,30,13,6,3,2,19,813.0,84.0,1016.37,233.72
1,1075,2016-01-15 01:00:00,30,15,1,4,1,2016,-14,-30,...,13,6,3,2,19,36,547.5,49.0,681.75,183.11
2,1075,2016-01-15 02:00:00,13,15,1,4,2,2016,-17,-31,...,6,3,2,19,36,78,237.0,17.0,289.67,71.99


In [25]:
# посчитаем средние по добавленным признакам
dt_new_feat['fare_amount_mean'] = dt_new_feat['fare_amount'] / dt_new_feat['val']
dt_new_feat['passenger_count_mean'] = dt_new_feat['passenger_count'] / dt_new_feat['val']
dt_new_feat['total_amount_mean'] = dt_new_feat['total_amount'] / dt_new_feat['val']
dt_new_feat['trip_distance_mean'] = dt_new_feat['trip_distance'] / dt_new_feat['val']
print dt_new_feat.shape
dt_new_feat.head(3) 

(445194, 70)


,cell_x,date_hour_x,val,day,month,weekday,hour,year,sh_1,sh_2,...,target_5,target_6,fare_amount,passenger_count,total_amount,trip_distance,fare_amount_mean,passenger_count_mean,total_amount_mean,trip_distance_mean
0,1075,2016-01-15 00:00:00,44,15,1,4,0,2016,-16,-59,...,2,19,813.0,84.0,1016.37,233.72,18.477273,1.909091,23.099318,5.311818
1,1075,2016-01-15 01:00:00,30,15,1,4,1,2016,-14,-30,...,19,36,547.5,49.0,681.75,183.11,18.250000,1.633333,22.725000,6.103667
2,1075,2016-01-15 02:00:00,13,15,1,4,2,2016,-17,-31,...,36,78,237.0,17.0,289.67,71.99,18.230769,1.307692,22.282308,5.537692


In [28]:
dt_new_feat.fillna(0,inplace=True)

In [29]:
# подготовлю тест и трейн
X_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]
X_train.drop(['weekday','val','date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]
X_test.drop(['weekday','val','date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)

C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [32]:
%%time
# Create linear regression object для для 6-ти концов истории своя модель
# заодно посчитаю ошибку за май
models = dict()
Qmay = 0 # ошибка предсказания за май
Qmay_mae = 0 # ошибка предсказания за май

for i in xrange(1,7):

    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]['target_'+str(i)].values
    y_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]['target_'+str(i)].values
    
    clf_LIN = linear_model.LinearRegression()
    clf_LIN.fit(X_train, y_train)
    pred_LIN = clf_LIN.predict(X_test)
    score_LIN = clf_LIN.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_LIN[pred_LIN < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_LIN)
    
    Qmay += sum(abs(y_test-pred_LIN))
    Qmay_mae += metrics.mean_absolute_error(y_test, pred_LIN)
    
    print 'точность модели linear regression для Т+'+str(i)+' = '+ str(score_LIN)+' Q=' + str(1/(102*739.*i)*Qmay) 
    print 'MAE = ' + str(Qmay_mae/i)    

точность модели linear regression для Т+1 = 0.97171376732 Q=20.8553231816
MAE = 20.8553231816
точность модели linear regression для Т+2 = 0.940781485885 Q=24.7564453386
MAE = 24.7564453386
точность модели linear regression для Т+3 = 0.920048744142 Q=27.3956722552
MAE = 27.3956722552
точность модели linear regression для Т+4 = 0.908087857172 Q=29.1992311789
MAE = 29.1992311789
точность модели linear regression для Т+5 = 0.903167094942 Q=30.4012866637
MAE = 30.4012866637
точность модели linear regression для Т+6 = 0.902236561992 Q=31.2089287214
MAE = 31.2089287214
Wall time: 18 s


In [33]:
# считаем ошибку прогноза  было 31.296, ошибка уменьшилсь очень незначительно
1/(102*739*6.)*Qmay

31.208928721418193

In [ ]:
#plt.figure(figsize=(16,10))
#plt.barh(X_test.columns, models[2][1].coef_, align='center',log=False)
#plt.show()

In [36]:
# подготовлю тест и трейн
X_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]
X_train.drop(['date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]
X_test.drop(['date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)

C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [37]:
%%time
# попробую RandomForest
Qmay_mae = 0 # ошибка предсказания за май
Qmay = 0 # ошибка предсказания за май

for i in xrange(1,7):

    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]['target_'+str(i)].values
    y_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]['target_'+str(i)].values
    
    clf_RF = ensemble.RandomForestRegressor(n_estimators=70,max_features=30,n_jobs=16)
    clf_RF.fit(X_train, y_train)
    pred_RF = clf_RF.predict(X_test)
    score_RF = clf_RF.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_RF[pred_RF < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_RF)
    
    Qmay += sum(abs(y_test-pred_RF))
    Qmay_mae += metrics.mean_absolute_error(y_test, pred_RF)  
    
    print 'точность модели linear regression для Т+'+str(i)+' = '+ str(score_RF)+' Q=' + str(1/(102*739.*i)*Qmay) 
    print 'MAE = ' + str(Qmay_mae/i)   

точность модели linear regression для Т+1 = 0.983555839894 Q=15.6774617831
MAE = 15.6774617831
точность модели linear regression для Т+2 = 0.975525617656 Q=17.0784796056
MAE = 17.0784796056
точность модели linear regression для Т+3 = 0.971943414865 Q=17.8994169702
MAE = 17.8994169702
точность модели linear regression для Т+4 = 0.970030239922 Q=18.4574783567
MAE = 18.4574783567
точность модели linear regression для Т+5 = 0.96842878245 Q=18.8606461529
MAE = 18.8606461529
точность модели linear regression для Т+6 = 0.967917702773 Q=19.1592253771
MAE = 19.1592253771
Wall time: 9min 10s


In [28]:
#19.9761247756 MAE для RndFor без доп признаков ошибка стала немного меньше

(295260, 57)

ок, посчитаем июнь

In [40]:
# для предсказания июня обучаться будем с 2014 по 05.2016
train_from = pd.to_datetime('2014-01-01 00:00:00')
train_to = pd.to_datetime('2016-05-31 23:00:00')

test_from = pd.to_datetime('2016-05-31 23:00:00')
test_to = pd.to_datetime('2016-06-30 17:00:00')

# подготовлю тест и трейн
X_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]
X_train.drop(['date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)
X_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]
X_test.drop(['date_hour_x','target_1','target_2','target_3','target_4','target_5','target_6'],
             inplace=True, axis=1)

# итоговый фрейм для кэгла
dt_itog = pd.DataFrame(dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & 
                                (dt_new_feat.date_hour_x <= test_to)][['date_hour_x','cell_x']])

C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\JB\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
%%time
# попробую RandomForest
Qmay_mae = 0 # ошибка предсказания за май
Qmay = 0 # ошибка предсказания за май

for i in xrange(1,7):

    # определяю целевые значения, т.е. сдвинутые на 1 - 6 часов
    y_train = dt_new_feat[(dt_new_feat.date_hour_x >= train_from) & (dt_new_feat.date_hour_x <= train_to)]['target_'+str(i)].values
    y_test = dt_new_feat[(dt_new_feat.date_hour_x >= test_from) & (dt_new_feat.date_hour_x <= test_to)]['target_'+str(i)].values
    
    clf_RF = ensemble.RandomForestRegressor(n_estimators=70,max_features=30,n_jobs=16)
    clf_RF.fit(X_train, y_train)
    pred_RF = clf_RF.predict(X_test)
    score_RF = clf_RF.score(X_test, y_test)
    
# занулим отриательные предсказания
    pred_RF[pred_RF < 0] = 0  # All low values set to 0
    
    models[i] = (i, clf_RF)
    
    Qmay += sum(abs(y_test-pred_RF))
    Qmay_mae += metrics.mean_absolute_error(y_test, pred_RF)  
    
    dt_itog[str(i)] = pred_RF
    
    print 'точность модели linear regression для Т+'+str(i)+' = '+ str(score_RF)+' Q=' + str(1/(102*715.*i)*Qmay) 
    print 'MAE = ' + str(Qmay_mae/i)   

точность модели linear regression для Т+1 = 0.981707174442 Q=15.608550665
MAE = 15.608550665
точность модели linear regression для Т+2 = 0.973995891979 Q=16.8698183189
MAE = 16.8698183189
точность модели linear regression для Т+3 = 0.97097765467 Q=17.5714178632
MAE = 17.5714178632
точность модели linear regression для Т+4 = 0.970373658679 Q=17.9974842315
MAE = 17.9974842315
точность модели linear regression для Т+5 = 0.96868333558 Q=18.3009177881
MAE = 18.3009177881
точность модели linear regression для Т+6 = 0.968461958469 Q=18.5355930671
MAE = 18.5355930671
Wall time: 11min 42s


In [49]:
# функция по приведения даты к нкжному формату
def date_str(timed):
    if (timed).strftime('%H')=='00': 
        a = str((timed).strftime('%Y-%m-%d_'))+'0' 
    else: a = str((timed).strftime('%Y-%m-%d_'))+str((timed).strftime('%H')).lstrip("0")
    return a

In [50]:
dt_itog.head(2)

,date_hour_x,cell_x,1,2,3,4,5,6
3311,2016-05-31 23:00:00,1075,26.142857,13.885714,5.928571,3.628571,2.300000,7.757143
3312,2016-06-01 00:00:00,1075,18.128571,8.800000,5.257143,4.914286,10.742857,28.042857


In [52]:
%%time
# соберем теперь файл для кэгла
arr_pred_june = [] # итоговый массив
dt_Kaggle = pd.DataFrame()
for i in range(dt_itog.shape[0]):
    for j in xrange(1,7):
        arr_pred_june.append([str(dt_itog.cell_x.iloc[i]) + '_' + date_str(dt_itog.date_hour_x.iloc[i]) + '_' + str(j),
                            int(round(dt_itog[str(j)].iloc[i]))])

Wall time: 21.7 s


In [53]:
all_pd = pd.DataFrame(arr_pred_june)
all_pd.columns = ['id','y']

In [55]:
print all_pd.shape
all_pd.head(2)

(437580, 2)


,id,y
0,1075_2016-05-31_23_1,26
1,1075_2016-05-31_23_2,14


In [56]:
all_pd.to_csv('itog6.csv', sep=',', index=False)

Score на kaggle стал лучше на 11% благодаря перехода от линейной регрессии на случайный лес, доп признаки тоже помогли примерно на 1%
https://yadi.sk/i/gWlyzanz3Zu8nu  вошел в топ 50 считаю неплохо =)

ссылка на решение https://www.kaggle.com/submissions/8089857/8089857.zip